In [4]:
import glob

In [3]:
import findspark
findspark.init("/Users/roongkun/spark-3.5.1-bin-hadoop3/")





In [5]:
spark_url = 'local'

In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StructType

In [7]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

24/05/03 02:11:48 WARN Utils: Your hostname, Roongkuns-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.188 instead (on interface en0)
24/05/03 02:11:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 02:11:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/03 02:11:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [61]:
base_path = "abstracts-retrieval-response"
for file in list(glob.glob("2023*.json")):
    try: 
        df = spark.read.option("multiline", True).json(file)

        is_authors_struct = isinstance(df.schema[base_path].dataType["authors"].dataType["author"].dataType, StructType)
        is_affiliations_struct = isinstance(df.schema[base_path].dataType["affiliation"].dataType, StructType)

        df = (df.withColumn("title", df[base_path + ".coredata.dc:title"])
        .withColumn("id", df[base_path + ".coredata.dc:identifier"])
        .withColumn("subject-area",F.col(base_path + ".subject-areas.subject-area.$"))
        .withColumn("year", F.col(base_path + ".item.bibrecord.head.source.publicationdate.year"))
        .withColumn("citation_count", F.col(base_path + ".coredata.citedby-count"))
        )

        if is_authors_struct:
            authors_array = F.array(df[base_path + ".authors.author"])
            df = df.withColumn("authors", authors_array)
        else:
            df = df.withColumn("authors", F.col(base_path + ".authors.author"))
        

        if is_affiliations_struct:
            affiliations_array = F.array(df[base_path + ".affiliation"])
            df = df.withColumn("affiliations", affiliations_array)
        else:
            df = df.withColumn("affiliations", F.col(base_path + ".affiliation"))
        


        df = df.drop(base_path)
        df.write.mode("append").json("c2023")
    except Exception as e:
        print(f"Error processing file {file}: {e}")

#Testing

In [8]:
df = spark.read.json("c2023/2023.json")

df.printSchema(False)

root
 |-- affiliations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- @href: string (nullable = true)
 |    |    |-- @id: string (nullable = true)
 |    |    |-- affiliation-city: string (nullable = true)
 |    |    |-- affiliation-country: string (nullable = true)
 |    |    |-- affilname: string (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- @_fa: string (nullable = true)
 |    |    |-- @auid: string (nullable = true)
 |    |    |-- @seq: string (nullable = true)
 |    |    |-- affiliation: string (nullable = true)
 |    |    |-- author-url: string (nullable = true)
 |    |    |-- ce:alias: string (nullable = true)
 |    |    |-- ce:alt-name: string (nullable = true)
 |    |    |-- ce:degrees: string (nullable = true)
 |    |    |-- ce:given-name: string (nullable = true)
 |    |    |-- ce:indexed-name: string (nullable = true)
 |    |    |-- ce:initials: string (nullable 

In [10]:
df.select(F.explode(df["affiliations.affilname"])).show()


+--------------------+
|                 col|
+--------------------+
|Chulalongkorn Uni...|
|Chulalongkorn Uni...|
|Center of Excelle...|
|Faculty of Medici...|
|King Chulalongkor...|
|Organisation Euro...|
|Rheinisch-Westfäl...|
|Tata Institute of...|
|Institut de Physi...|
|Institut Pluridis...|
|Faculty of Scienc...|
|State Key Laborat...|
|Purdue University...|
|Rutgers Universit...|
|Academy of Scient...|
|Istituto Nazional...|
|Istituto Nazional...|
|Istituto Nazional...|
|Istituto Nazional...|
|Istituto Nazional...|
+--------------------+
only showing top 20 rows

